##import library

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei'] 
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import make_scorer, roc_curve, roc_auc_score
# from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

##import dataset

In [ ]:
dataset_all = pd.read_csv("output.csv")
# test_set = pd.read_csv("test_data.csv")

##set category and data size

In [ ]:
interested_categories =  [ "影劇", "政治", "健康", "時尚", "生活", "地方"]

dataset = dataset_all[dataset_all['category'].isin(interested_categories)]

dataset = dataset.groupby('category').head(500)
# dataset = dataset.sample(n=2000, random_state=42)
# num_samples_needed = 1000
# random_samples = dataset.sample(n=num_samples_needed, replace=True, random_state=42)

# dataset = pd.concat([dataset, random_samples], ignore_index=True)

dataset.reset_index(drop=True, inplace=True)


##imbalanced dataset

In [ ]:
# # dataset = dataset_all.sample(n=1800, random_state=42)
# # dataset = dataset_all.groupby('category').apply(lambda x: x.sample(n=300, random_state=42)).reset_index(drop=True)
# # dataset = dataset_all.groupby('category').head(500)
# # dataset.reset_index(drop=True, inplace=True)

# interested_categories = ["影劇", "體育", "政治", "財經", "健康", "時尚"]

# dataset = dataset_all[dataset_all['category'].isin(interested_categories)]

# samples_per_category = {
#     "影劇": 40,
#     "體育": 224,
#     "政治": 83,
#     "財經": 485,
#     "健康": 177,
#     "時尚": 164
    
# }

# sampled_data = []
# for category, num_samples in samples_per_category.items():
#     category_data = dataset[dataset['category'] == category]
#     sampled_data.append(category_data.sample(n=num_samples, random_state=42))

# dataset = pd.concat(sampled_data)

# dataset.reset_index(drop=True, inplace=True)

##print to show the dataset  /  print to show the first text of dataset  /  shape of dataset

In [ ]:
print(dataset.head())
print(dataset['title'][0])
print(dataset.shape)

##check information of columns of dataset


In [ ]:
dataset.info()

##count values of given categories

In [ ]:
dataset['category'].value_counts()

##convert categories name into numerical index

In [ ]:
# Associate Category names with numerical index and save it in new column CategoryId
target_category = dataset['category'].unique()
print(target_category)

##to show new column were added

In [ ]:
dataset['CategoryId'] = dataset['category'].factorize()[0]
# dataset.head()

##show category id in dictionary form

In [ ]:
# Create a dictionary ( python datastructure - like a lookup table) that 
# can easily convert category names into category_ids and vice-versa
category = dataset[['category', 'CategoryId']].drop_duplicates().sort_values('CategoryId')
category_to_id = dict(category.values)
id_to_category = dict(category[['CategoryId', 'category']].values)
id_to_category

##visualizing category data in numeric form

In [ ]:
# 長條圖
dataset.groupby('category').CategoryId.value_counts().plot(kind = "bar", color = ["pink", "orange", "red", "yellow", "lightblue", "lightgreen", "lightcoral", "lightskyblue", "lightseagreen", "mediumorchid"])
plt.xlabel("Category of data")
plt.title("Visulaize numbers of Category of data")
plt.show()

##print category coulmn of dataset

In [ ]:
text = dataset["title"]
print(text.head(5))
category = dataset['category']
category.head(5)

##remove special characters

In [ ]:
def special_char(text):
  reviews = ''
  for x in text:
    if x.isalnum():
      reviews = reviews + x
    else:
      reviews = reviews + ' '
  return reviews

dataset['title'] = dataset['title'].apply(special_char)
dataset['title'][1]

##convert everything in lower case

In [ ]:
def convert_lower(text):
   return text.lower()

dataset['title'] = dataset['title'].apply(convert_lower)
dataset['title'][1]

##after text_preprocessing print dataset

In [ ]:
dataset

##declared dependent and independent value

In [ ]:
x = dataset['title']
y = dataset['category']

##create and fit bag of words model

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import jieba

def chinese_tokenizer(text):
    words = jieba.cut(text)
    # print(" ".join(words))
    return " ".join(words)

def dummy_tokenizer(text):
    print(text)
    return text

# tokenizer
cv = CountVectorizer(tokenizer=chinese_tokenizer,token_pattern=None, max_features=1000)

# no tokenizer
# cv = CountVectorizer(tokenizer=dummy_tokenizer, max_features=500)

x = cv.fit_transform(dataset.title).toarray()
x = cv.fit_transform(dataset.title).toarray()

print("X.shape = ", x.shape)
print("y.shape = ", y.shape)


In [ ]:
print(x)

##train test and split the dataset into training and test set

In [ ]:
# from sklearn.model_selection import train_test_split
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0, shuffle = True)
# print(len(x_train))
# print(len(x_test))

##create empty list

In [ ]:
#create list of model and accuracy dicts
perform_list = []

##create, fit and predict all Machine learning model

In [ ]:
from sklearn.metrics import silhouette_score
import seaborn as sns

def plot_confusion_matrix(cm, target_names, title='Confusion matrix', cmap=None, normalize=True, name=None):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plt.figure(num= name, figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt=".2f", cmap=cmap, xticklabels=target_names, yticklabels=target_names)
    plt.title(title)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()


def run_model(model_name):
    mdl=''
    if model_name == 'Logistic Regression':
        mdl = LogisticRegression()
    elif model_name == 'Random Forest':
        mdl = RandomForestClassifier(n_estimators=100 ,criterion='entropy' , random_state=0)
    elif model_name == 'Multinomial Naive Bayes':
        mdl = MultinomialNB(alpha=1.0,fit_prior=True)
    elif model_name == 'Support Vector Classifer':
        mdl = SVC()
    elif model_name == 'K Nearest Neighbour':
        mdl = KNeighborsClassifier(n_neighbors=20 , metric= 'minkowski' , p = 4)
    elif model_name == 'K-Means Clustering':
        mdl = KMeans(n_clusters=6, random_state=0)
   
        
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(mdl, x, y, cv=kfold)
    best_acc = 0
        
    if mdl != '':
        if model_name == 'K-Means Clustering':
            mdl.fit(x) 
            inertia = mdl.inertia_
            silhouette = silhouette_score(x, mdl.labels_)
            print(f'Inertia: {inertia}')
            print(f'Silhouette Score: {silhouette}')
                
        else:
            for i, (train_index, test_index) in enumerate(kfold.split(x, y)):
                X_train_fold, X_test_fold = x[train_index], x[test_index]
                y_train_fold, y_test_fold = y[train_index], y[test_index]
                
                mdl.fit(X_train_fold, y_train_fold)
                
                y_pred = mdl.predict(X_test_fold)
                
                accuracy = accuracy_score(y_test_fold, y_pred)
                precision = precision_score(y_test_fold, y_pred, average='micro')
                recall = recall_score(y_test_fold, y_pred, average='micro')
                f1 = f1_score(y_test_fold, y_pred, average='micro')
                
                if accuracy > best_acc:
                    best_acc = accuracy
                    x_train = X_train_fold
                    x_test = X_test_fold
                    y_train = y_train_fold
                    y_test = y_test_fold
                    
                
                perform_list.append({
                    'Model': model_name,
                    'Fold': i + 1,
                    'Accuracy': round(accuracy, 2),
                    'Precision': round(precision, 2),
                    'Recall': round(recall, 2),
                    'F1': round(f1, 2)
                })
                
            mdl.fit(x_train, y_train)
            y_pred = mdl.predict(x_test)
            score = accuracy_score(y_test, y_pred)
            print("accuracy of "+model_name+" is:",score)
            print(classification_report(y_pred, y_test, target_names=target_category))
            cm = confusion_matrix(y_test, y_pred)
           
            plot_confusion_matrix(cm, target_names=target_category, title='Confusion Matrix', cmap='tab20', name = model_name)

##Logistic Regression

In [ ]:
run_model('Logistic Regression')

##Random Forest

In [ ]:
run_model('Random Forest')

##Multinomial Naive Bayes

In [ ]:
run_model('Multinomial Naive Bayes')

##Support vector machine

In [ ]:
run_model('Support Vector Classifer')

##KNN

In [ ]:
run_model('K Nearest Neighbour')

##K-Means

In [ ]:
run_model('K-Means Clustering')

##best model to perform accuracy score

In [ ]:

perform_df = pd.DataFrame(perform_list)
# Group by 'Model' and find the row with the highest accuracy for each model
best_fold_df = perform_df.loc[perform_df.groupby('Model')['Accuracy'].idxmax()]
print(best_fold_df)

##create dataframe of model, accuracy, precision,recall and f1

In [ ]:
perform_list = pd.DataFrame(data=perform_list)
# perform_list = perform_list[['Model', 'Test Accuracy', 'Precision', 'Recall', 'F1']]
perform_list